In [1]:
%%capture
!pip install --upgrade pip
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio
!pip install pyannote.audio
!pip install jiwer
!pip install peft

In [2]:
%%capture
# Install cpu/cuda pytorch (>=1.9) dependency from pytorch.org, e.g.:
!pip install torch torchaudio -f https://download.pytorch.org/whl/cpu/torch_stable.html
# Install DeepFilterNet
!pip install deepfilternet
# Or install DeepFilterNet including data loading functionality for training (Linux only)
!pip install deepfilternet[train]

In [3]:
from datasets import Dataset, DatasetDict, concatenate_datasets
import os
import torch, torchaudio
import tqdm as tqdm
from IPython.display import Audio

# HuggingFace Login

In [ ]:
from huggingface_hub import login

login(token="*")

# PREPROCESSING

Define const

In [5]:
# Define paths and parameters
DATA_DIR = "/kaggle/input/vlsp-trainset/vlsp2020_train_set_02/"
MODEL_NAME = "openai/whisper-medium"
MAX_SAMPLES = 10000
TRAIN_SIZE = 0.8
VAL_SIZE = 0.1
TEST_SIZE = 0.1

Load model

In [6]:
from transformers import WhisperForConditionalGeneration, AutoModelForSpeechSeq2Seq

model = AutoModelForSpeechSeq2Seq.from_pretrained(MODEL_NAME)
model.generation_config.language = "vietnamese"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

2025-05-18 02:36:21.262454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747535781.435412      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747535781.485947      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

Noise Removing

In [7]:
def show_audio(audio, sr, audio_label):
  print(audio_label)
  display(Audio(audio, rate=sr))

In [8]:
from df.enhance import enhance, init_df, load_audio, save_audio
import torch

TARGET_SR = 16000

df_model, df_state, _ = init_df()
resampler = torchaudio.transforms.Resample(orig_freq=df_state.sr(), new_freq=TARGET_SR)

def enhance_waveform(noisy_path):
    audio, sr = load_audio(noisy_path, sr=df_state.sr())
    enhanced = enhance(df_model, df_state, audio)
    #tmp_path = "/kaggle/working/tmp_sample.wav"
    #save_audio(tmp_path, enhanced, df_state.sr())
    resampled = resampler(enhanced)
    return resampled, TARGET_SR

df_state.sr()

/usr/local/lib/python3.11/dist-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData


2025-05-18 02:36:48 | INFO     | DF | Running on torch 2.6.0+cu124
2025-05-18 02:36:48 | INFO     | DF | Running on host 0ef4998ca373
2025-05-18 02:36:48 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-05-18 02:36:48 | INFO     | DF | Using DeepFilterNet3 model at /root/.cache/DeepFilterNet/DeepFilterNet3
2025-05-18 02:36:48 | INFO     | DF | Initializing model `deepfilternet3`


fatal: not a git repository (or any of the parent directories): .git


2025-05-18 02:36:48 | INFO     | DF | Found checkpoint /root/.cache/DeepFilterNet/DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2025-05-18 02:36:48 | INFO     | DF | Running on device cuda:0
2025-05-18 02:36:48 | INFO     | DF | Model loaded


48000

In [9]:
#Test
#audio_files, transcript_files = get_audio_files()

enhanced, sr = enhance_waveform("/kaggle/input/vlsp-trainset/vlsp2020_train_set_02/database_sa1_Jan08_Mar19_cleaned_utt_0000000045-1.wav")
print("Enhanced: ",enhanced)
print(sr)
print("Enhanced Size: ", enhanced.shape)
show_audio(enhanced, sr, "Enhanced Audio:")

print("="*20)
waveform, sample_rate = torchaudio.load("/kaggle/input/vlsp-trainset/vlsp2020_train_set_02/database_sa1_Jan08_Mar19_cleaned_utt_0000000045-1.wav")
print("Waveform: ",waveform)
print(sample_rate)
print("Waveform Size: ", waveform.shape)
show_audio(waveform, sample_rate, "Original Audio:")

2025-05-18 02:36:48 | WARNING  | DF | Audio sampling rate does not match model sampling rate (16000, 48000). Resampling...
/usr/local/lib/python3.11/dist-packages/df/io.py:106: UserWarning: "sinc_interpolation" resampling method name is being deprecated and replaced by "sinc_interp_hann" in the next release. The default behavior remains unchanged.
  return ta_resample(audio, orig_sr, new_sr, **params)


Enhanced:  tensor([[-3.8023e-03, -4.5763e-03, -4.7790e-03,  ...,  6.0069e-05,
          5.2587e-05,  6.1459e-05]])
16000
Enhanced Size:  torch.Size([1, 180320])
Enhanced Audio:


Waveform:  tensor([[-0.0194, -0.0195, -0.0185,  ...,  0.0219,  0.0204,  0.0237]])
16000
Waveform Size:  torch.Size([1, 180320])
Original Audio:


Create DatasetDict

# Test

In [10]:
!pip3 install https://github.com/kpu/kenlm/archive/master.zip

     - 553.6 kB 12.0 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp311-cp311-linux_x86_64.whl size=3185087 sha256=7a47f5c983f27c4366801555e67e9c6f43435659c6b8c972673f8c3688a77e89
  Stored in directory: /tmp/pip-ephem-wheel-cache-gmhhfllz/wheels/4e/ca/6a/e5da175b1396483f6f410cdb4cfe8bc8fa5e12088e91d60413
Successfully built kenlm


In [11]:
import kenlm
import heapq
from transformers import pipeline, WhisperProcessor, AutoProcessor, AutoModelForSpeechSeq2Seq
import torch.nn.functional as F
import re

In [12]:
SPECIAL_TOKENS = {"<|startoftranscript|>", "<|vi|>", "<|transcribe|>", "<|notimestamps|>"}

def strip_special_tokens(seq):
    return " ".join([tok for tok in seq.split() if tok not in SPECIAL_TOKENS])

def last_word_has_vietnamese_vowel(text):
    vietnamese_vowels = "aăâeêioôơuưyAĂÂEÊIOÔƠUƯYàáảãạằắẳẵặèéẻẽẹềếểễệìíỉĩịòóỏõọồốổỗộờớởỡợùúủũụừứửữựỳýỷỹỵ"
    
    # Tách các từ bằng khoảng trắng và lấy từ cuối
    words = text.strip().split()
    if not words:
        return False
    
    last_word = words[-1]
    if last_word == "gi":
        return false
    else: return any(char in vietnamese_vowels for char in last_word) 

In [13]:
confusable_onsets = [
    ['l', 'n'], ['d', 'gi', 'r'], ['ch', 'tr'], ['đ', 't']
]
confusable_codas = [
    ['n', 'ng'], ['t', 'c']
]
confusable_vowels = [
    # ['a', 'ă', 'â'], ['e', 'ê'], ['o', 'ô', 'ơ'], ['i', 'y'],
    # ['ai', 'ay'], ['ao', 'au'], ['âu', 'ău'], ['ia', 'ya'],
    # ['iêu', 'yêu'], ['ươi', 'uơi'], ['oai', 'oay']
]

# Tạo bản đồ âm sang nhóm
def make_confusion_map(groups):
    return {variant: group for group in groups for variant in group}

onset_map = make_confusion_map(confusable_onsets)
coda_map = make_confusion_map(confusable_codas)
vowel_map = make_confusion_map(confusable_vowels)

# Hàm lấy nhóm âm đầu
def get_onset_group(word):
    for k in sorted(onset_map.keys(), key=lambda x: -len(x)):
        if word.startswith(k):
            return k, onset_map[k]
    return None, None

# Hàm lấy nhóm âm cuối
def get_coda_group(word):
    for k in sorted(coda_map.keys(), key=lambda x: -len(x)):
        if word.endswith(k):
            return k, coda_map[k]
    return None, None

# Hàm lấy nhóm nguyên âm ở giữa
def get_vowel_group(word):
    for k in sorted(vowel_map.keys(), key=lambda x: -len(x)):
        if k in word:
            return k, vowel_map[k]
    return None, None

# Sinh biến thể của từ
def generate_word_variants(word):
    variants = set()
    variants.add(word)

    onset, onset_group = get_onset_group(word)
    if onset:
        for alt in onset_group:
            if alt != onset:
                variants.add(alt + word[len(onset):])

    coda, coda_group = get_coda_group(word)
    if coda:
        for alt in coda_group:
            if alt != coda:
                variants.add(word[:-len(coda)] + alt)

    vowel, vowel_group = get_vowel_group(word)
    if vowel:
        for alt in vowel_group:
            if alt != vowel:
                variants.add(word.replace(vowel, alt))

    return variants

# Beam search kết hợp LM để sửa lỗi
def beam_search_correction(sentence, lm_model, beam_width=5):
    words = sentence.strip().split()
    beam = [("", 0.0)]  # (prefix sentence, LM score)

    for word in words:
        new_beam = []
        word_variants = generate_word_variants(word)

        for prefix, score in beam:
            for variant in word_variants:
                new_sent = (prefix + " " + variant).strip()
                new_score = lm_model.score(new_sent, bos=True, eos=True)
                new_beam.append((new_sent, new_score))

        beam = sorted(new_beam, key=lambda x: x[1], reverse=True)[:beam_width]

    return beam[0][0], beam[0][1]

def remove_trailing_punctuation(sentence):
    return re.sub(r"[.?!…]+$", "", sentence.strip())

In [14]:
lm = kenlm.Model("/kaggle/input/n-gram/pytorch/default/1/vi_lm_4grams.bin")

input_sentence = "chả lại"  # câu sai
corrected, score = beam_search_correction(input_sentence, lm, beam_width=10)

print("Câu gốc:", input_sentence)
print("Câu sửa:", corrected)
print("Điểm LM:", score)

Câu gốc: chả lại
Câu sửa: trả lại
Điểm LM: -6.329550266265869


In [15]:
import os

base_dir = "/kaggle/input/private-test-wav/private-test-data-asr"

audio_files = [
    os.path.join(base_dir, f"audio_{i:04d}.wav")
    for i in range(1, 4269)  # từ 1 đến 4268
]

In [16]:
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

transcriber = pipeline("automatic-speech-recognition", model="vinai/PhoWhisper-medium")

output_lines = []
for audio_path in tqdm(audio_files, desc="Transcribing audio files"):
    result = transcriber(audio_path)
    transcript = result['text'].strip()
    transcript = remove_trailing_punctuation(transcript)

    #transcript, score = beam_search_correction(transcript, lm, beam_width=5)

    output_lines.append(transcript + '\n')
    
# Ghi ra transcripts.txt
with open('/kaggle/working/transcripts.txt', 'w', encoding='utf-8') as f:
    f.writelines(output_lines)

print("✅ Đã lưu transcripts.txt thành công")

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.72k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Device set to use cuda:0
Transcribing audio files: 100%|██████████| 4268/4268 [1:23:58<00:00,  1.18s/it]

✅ Đã lưu transcripts.txt thành công
